In [40]:
#### Loading the packages to be used
from __future__ import print_function  # Python 2 and 3
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
from scipy import stats
import os
%matplotlib inline
plt.style.use('ggplot')
np.random.seed(0)

In [41]:
from sklearn import linear_model
ols = linear_model.LinearRegression()

In [42]:
os.chdir("/Users/ptero793/Desktop/house_prices")
data = pd.read_csv("train.csv")

In [43]:
data.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [44]:
data.BldgType.value_counts()

1Fam      1220
TwnhsE     114
Duplex      52
Twnhs       43
2fmCon      31
Name: BldgType, dtype: int64

In [45]:
df=data.copy()

Adding Indoor Square footage together to get full house SF (excluding outdoor)

In [46]:
df['Total_size']= df['BsmtFinSF1']+ df['BsmtFinSF2'] + df['1stFlrSF'] +df['2ndFlrSF']+df['GarageArea']



In [47]:
#create a helper for bldgtype
def helper(x):
    if x.BldgType == "TwnhsE":
        return 'Twnhs'
    else:
        return x.BldgType

In [48]:
def helper(x):
    if x.BldgType == "TwnhsE":
        return 'Twnhs'
    else:
        return x.BldgType

df['BldgType_clean']=df.apply(helper,axis=1)



In [49]:
df[['LotArea','Total_size','WoodDeckSF','OpenPorchSF','PoolArea']]

,LotArea,Total_size,WoodDeckSF,OpenPorchSF,PoolArea
0,8450,2964,0,61,0
1,9600,2700,298,0,0
2,11250,2880,0,42,0
3,9550,2575,0,35,0
4,14260,3689,192,84,0
...,...,...,...,...,...
1455,7917,2107,0,40,0
1456,13175,3526,349,0,0
1457,9042,2867,0,60,0
1458,9717,2396,366,0,0


In [50]:
df['Outdoor_Porch_Size']= df['WoodDeckSF']+df['OpenPorchSF']


Adding helper functions to combine 2 columns (remodel year,year built) to redefine "age_at_sale" - a single column to account for house at the time of sale. We subtract the most recent year of construction from the year of sale to get the homes age in years

In [51]:
# #helper_1
# def recent_constr(x):
#     if(x.YearBuilt < x.YearRemodAdd):
#         return x.YearRemodAdd
#     else: 
#         return x.YearBuilt

# #defining intermediate column    
# df['time_remod'] = data.apply(recent_constr,axis=1)

#helper 2
def recent_constr(x):
    if(x.YearBuilt < x.YearRemodAdd):
        return x.YearRemodAdd
    else: 
        return x.YearBuilt

#defining intermediate column 2    
df['Recent_yr'] = df.apply(recent_constr,axis=1)

#time to sale helper defined:
def time_to_sale(x):
    return x.YrSold - x.Recent_yr

#new column defined to be used in Train
df['Age_at_sale']=df.apply(time_to_sale,axis=1)

Taking log Transform of target variable (SalePrice):

In [52]:
df['log_sales']=np.log(df['SalePrice'])



In [53]:
model2_df = df[['Total_size','LotArea','Neighborhood','BldgType_clean','OverallQual','Age_at_sale','ExterQual',
                'Outdoor_Porch_Size','YrSold']]

In [54]:
model2_df.columns


Index(['Total_size', 'LotArea', 'Neighborhood', 'BldgType_clean',
       'OverallQual', 'Age_at_sale', 'ExterQual', 'Outdoor_Porch_Size',
       'YrSold'],
      dtype='object')

In [55]:
model2_df.head()


,Total_size,LotArea,Neighborhood,BldgType_clean,OverallQual,Age_at_sale,ExterQual,Outdoor_Porch_Size,YrSold
0,2964,8450,CollgCr,1Fam,7,5,Gd,61,2008
1,2700,9600,Veenker,1Fam,6,31,TA,298,2007
2,2880,11250,CollgCr,1Fam,7,6,Gd,42,2008
3,2575,9550,Crawfor,1Fam,7,36,TA,35,2006
4,3689,14260,NoRidge,1Fam,8,8,Gd,276,2008


In [56]:
from sklearn.preprocessing import OneHotEncoder

In [57]:
model2_df.columns

Index(['Total_size', 'LotArea', 'Neighborhood', 'BldgType_clean',
       'OverallQual', 'Age_at_sale', 'ExterQual', 'Outdoor_Porch_Size',
       'YrSold'],
      dtype='object')

One Hot Encoding for categorical variable

In [58]:
# dummify Neighborhood column
Neighborhood_ohe = OneHotEncoder( drop='first', sparse = False )
Neighborhood_enc = Neighborhood_ohe.fit_transform( model2_df[['Neighborhood']] )
Neighborhood_enc = pd.DataFrame( Neighborhood_enc, columns=Neighborhood_ohe.get_feature_names(['Neighborhood']) )
model2_df = pd.concat( (model2_df.drop(["Neighborhood"], axis=1).reset_index(drop = True), Neighborhood_enc), axis = 1 )

#dummify Building Type

BldgType_clean_ohe = OneHotEncoder( drop='first', sparse = False )
BldgType_clean_enc = BldgType_clean_ohe.fit_transform( model2_df[['BldgType_clean']] )
BldgType_clean_enc = pd.DataFrame( BldgType_clean_enc, columns=BldgType_clean_ohe.get_feature_names(['BldgType_clean']) )
model2_df = pd.concat( (model2_df.drop(["BldgType_clean"], axis=1).reset_index(drop = True), BldgType_clean_enc), axis = 1 )


#dummify ExterQual

ExterQual_ohe = OneHotEncoder( drop='first', sparse = False )
ExterQual_enc = ExterQual_ohe.fit_transform( model2_df[['ExterQual']] )
ExterQual_enc = pd.DataFrame( ExterQual_enc, columns=ExterQual_ohe.get_feature_names(['ExterQual']) )
model2_df = pd.concat( (model2_df.drop(["ExterQual"], axis=1).reset_index(drop = True), ExterQual_enc), axis = 1 )



#dummify YrSold

YrSold_ohe = OneHotEncoder( drop='first', sparse = False )
YrSold_enc = YrSold_ohe.fit_transform( model2_df[['YrSold']] )
YrSold_enc = pd.DataFrame( YrSold_enc, columns=YrSold_ohe.get_feature_names(['YrSold']) )
model2_df = pd.concat( (model2_df.drop(["YrSold"], axis=1).reset_index(drop = True), YrSold_enc), axis = 1 )


In [62]:
model2_df.shape

(1460, 39)

In [63]:
X=model2_df

In [66]:
y=df.log_sales

In [67]:
#try to keep interpretable
from sklearn.model_selection import train_test_split
Xtrain, Xtest,ytrain, ytest = train_test_split(X,y,test_size = 0.2)

In [68]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()

#below learning from training set which is why we use fit
Xtrain = pd.DataFrame(ss.fit_transform(Xtrain),columns = Xtrain.columns)

In [69]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

In [76]:
lasso = Lasso()
params = {'alpha':(0.0000001,0.0001,0.001,0.01,0.1,0.5,1,2,10,100,10000),'fit_intercept':[True,False]}
grid_search_lasso = GridSearchCV( estimator = lasso, param_grid = params)
grid_search_lasso.fit(Xtrain,ytrain)

GridSearchCV(cv=None, error_score=nan,
             estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=1000, normalize=False, positive=False,
                             precompute=False, random_state=None,
                             selection='cyclic', tol=0.0001, warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'alpha': (1e-07, 0.0001, 0.001, 0.01, 0.1, 0.5, 1, 2,
                                   10, 100, 10000),
                         'fit_intercept': [True, False]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [77]:
grid_search_lasso.best_params_

{'alpha': 0.0001, 'fit_intercept': True}

In [78]:
grid_search_lasso.best_score_

0.8666675973910936

In [79]:
lasso = Lasso(alpha=.0001)
lasso.fit(Xtrain, ytrain).coef_

array([ 0.157616  ,  0.02247124,  0.13060942, -0.04954206,  0.02189818,
       -0.00559984, -0.02517897, -0.03071611, -0.00674644, -0.02610023,
        0.00504045, -0.05406345, -0.01111557, -0.05547745, -0.0271745 ,
       -0.02696206, -0.04786395, -0.00486544, -0.01874687, -0.00331501,
        0.00741702, -0.06518141, -0.01899085, -0.03789918, -0.02701518,
       -0.00481791,  0.00389278, -0.01422626, -0.00098863, -0.00728161,
       -0.00657065, -0.03334595, -0.01555128, -0.01011009, -0.02708725,
        0.00385237,  0.00229419, -0.00479505,  0.00268682])

In [80]:
lasso.fit(Xtrain, ytrain).coef_
lst = []
for i in (list(zip(lasso.fit(Xtrain, ytrain).coef_, Xtrain.columns))):
    lst.append(i)
lst

[(0.15761600379616791, 'Total_size'),
 (0.022471240883708207, 'LotArea'),
 (0.13060942341266374, 'OverallQual'),
 (-0.04954206394037876, 'Age_at_sale'),
 (0.02189818376720106, 'Outdoor_Porch_Size'),
 (-0.005599838909560209, 'Neighborhood_Blueste'),
 (-0.025178968830040123, 'Neighborhood_BrDale'),
 (-0.03071610875815192, 'Neighborhood_BrkSide'),
 (-0.006746441756859876, 'Neighborhood_ClearCr'),
 (-0.026100229358603, 'Neighborhood_CollgCr'),
 (0.0050404524697656005, 'Neighborhood_Crawfor'),
 (-0.054063453125305826, 'Neighborhood_Edwards'),
 (-0.011115571661968372, 'Neighborhood_Gilbert'),
 (-0.05547745213814123, 'Neighborhood_IDOTRR'),
 (-0.027174495799701485, 'Neighborhood_MeadowV'),
 (-0.026962058186501374, 'Neighborhood_Mitchel'),
 (-0.0478639460337705, 'Neighborhood_NAmes'),
 (-0.004865438035460248, 'Neighborhood_NPkVill'),
 (-0.01874686701958918, 'Neighborhood_NWAmes'),
 (-0.0033150148344491736, 'Neighborhood_NoRidge'),
 (0.007417018415000081, 'Neighborhood_NridgHt'),
 (-0.065181411

In [81]:
print('R^2 is equal to %.3f' %(lm.score(X, Y)))
print('RSS is equal to %.3f' %(np.sum((Y-lm.predict(X))**2)))
print('The intercept is %.3f' %(lm.intercept_))
print('The slopes are %s' %(lm.coef_))

NameError: name 'lm' is not defined